# PySpark Quick Start

In [ ]:
# import necessary libraries
import pandas as pd 
import numpy
import matplotlib.pyplot as plt 
from pyspark.sql import SparkSession

# create sparksession
spark = SparkSession \
    .builder \
    .appName("Pysparkexample") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
text_file = spark.read.text("README.md")
text_file.count()

92

In [3]:
text_file.first()

Row(value='# PySpark Tutorial')

In [2]:
!which python

/opt/conda/bin/python
